In [8]:

!pip install -U datasets

In [1]:
# 최종 통합 코드 (Colab friendly: 학습 + 시각화)

#참고코드 https://github.com/datnnt1997/multi-head_self-attention/blob/master/SelfAttention.ipynb
#이론 참조 깃허브 https://github.com/IAAR-Shanghai/Awesome-Attention-Heads


import torch                      # PyTorch 핵심 모듈
import torch.nn as nn             # 신경망 구성용 모듈
import torch.nn.functional as F  # 활성함수 등
import pandas as pd              # 표 형태 출력을 위한 모듈
from transformers import GPT2Tokenizer  # 토큰화를 위한 HuggingFace GPT2 tokenizer
from torch.utils.data import DataLoader, Dataset  # 데이터셋 관리
import random

# GPU 설정 (가능하면 CUDA 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 어텐션 계산 함수: scaled dot-product attention
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size(-1)
    scores = torch.matmul(q, k.transpose(-2, -1)) / d_k**0.5  # 유사도 점수 계산 후 scaling
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    attn = F.softmax(scores, dim=-1)  # attention 가중치
    output = torch.matmul(attn, v)    # 가중합 결과 출력
    return output, attn

# 멀티헤드 셀프 어텐션 클래스 정의
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0  # head 수가 나누어 떨어져야 함
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        # Q, K, V를 위한 선형변환
        self.q_linear = nn.Linear(embed_dim, embed_dim)
        self.k_linear = nn.Linear(embed_dim, embed_dim)
        self.v_linear = nn.Linear(embed_dim, embed_dim)
        self.fc_out = nn.Linear(embed_dim, embed_dim)  # 출력 결합

    def forward(self, x):
        B, T, C = x.size()  # 배치크기, 시퀀스길이, 임베딩 차원
        # Q, K, V 생성 후 head 차원 추가
        Q = self.q_linear(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.k_linear(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.v_linear(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        # 어텐션 수행
        out, attn = scaled_dot_product(Q, K, V)
        # 다시 원래 차원으로 복원
        out = out.transpose(1, 2).contiguous().view(B, T, C)
        return self.fc_out(out), attn

# Transformer 스타일의 작은 언어모델
class TinyTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, num_heads=4):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, embed_dim)  # 단어 임베딩
        self.attn = MultiHeadSelfAttention(embed_dim, num_heads)  # 셀프 어텐션
        self.norm = nn.LayerNorm(embed_dim)   #정규화
        self.lm_head = nn.Linear(embed_dim, vocab_size)  # 출력 예측 (language modeling head)

    def forward(self, x):
        x = self.token_emb(x)  # [B, T, C]
        x_norm = self.norm(x)               # 정규화
        x_attn, attn = self.attn(x_norm)    # 어텐션 결과와 가중치
        x = x + x_attn                      #  Residual 연결
        logits = self.lm_head(x)  # 다음 토큰 예측

        return logits, attn

from datasets import load_dataset

# TinyStories 데이터셋 직접 불러오기 (샘플 50000개만 사용)
dataset = load_dataset("roneneldan/TinyStories", split="train[:50000]")
# 텍스트 추출
text = "\n".join(example["text"] for example in dataset)




# GPT2 토크나이저 사용
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # pad token 설정 (eos로 대체)
# 텍스트를 token ID로 인코딩
encodings = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=64)
input_ids = encodings["input_ids"]  # [num_sentences, seq_len]

'''
from collections import defaultdict
from torch.nn.utils.rnn import pad_sequence

# merge 함수: (a,b) 쌍을 new_index로 묶기
def merge(indices, pair, new_index):
    merged = []
    i = 0
    while i < len(indices):
        if i < len(indices) - 1 and (indices[i], indices[i+1]) == pair:
            merged.append(new_index)
            i += 2
        else:
            merged.append(indices[i])
            i += 1
    return merged

# BPE 학습 함수
def train_bpe(text: str, num_merges: int):
    indices = list(text.encode("utf-8"))
    merges = {}
    vocab = {i: bytes([i]) for i in range(256)}

    for i in range(num_merges):
        counts = defaultdict(int)
        for a, b in zip(indices, indices[1:]):
            counts[(a, b)] += 1
        if not counts:
            break
        pair = max(counts, key=counts.get)
        new_index = 256 + i
        merges[pair] = new_index
        vocab[new_index] = vocab[pair[0]] + vocab[pair[1]]
        indices = merge(indices, pair, new_index)

    return {"vocab": vocab, "merges": merges}

# BPE 토크나이저 클래스
class BPETokenizer:
    def __init__(self, vocab, merges):
        self.vocab = vocab            # {int: bytes} 형태
        self.merges = merges          # {(a,b): new_index}
        self.inverse_vocab = {v: k for k, v in vocab.items()}  # bytes -> int 역매핑

    def encode(self, text: str):
        ids = list(text.encode("utf-8"))
        for (a, b), new_id in self.merges.items():
            ids = merge(ids, (a, b), new_id)
        return ids

    def decode(self, ids: list[int]):
        return b"".join([self.vocab[i] for i in ids]).decode("utf-8", errors="ignore")

# -----------------------
# 여기부터 GPT2Tokenizer 대체 부분

# 1) BPE 학습 (num_merges는 적절히 조절)
tokenizer_params = train_bpe(text, num_merges=1000)

# 2) 토크나이저 객체 생성
tokenizer = BPETokenizer(**tokenizer_params)

# 3) 텍스트 줄 단위로 토크나이징 및 텐서화
lines = text.split("\n")
tokenized_ids = [torch.tensor(tokenizer.encode(line)) for line in lines]

# 4) 패딩 (패딩 토큰 ID는 0으로 지정)
input_ids = pad_sequence(tokenized_ids, batch_first=True, padding_value=0)  # shape: [num_sentences, seq_len]

# 이제 input_ids를 기존 GPT2Tokenizer 대체해서 학습에 바로 사용 가능
'''

# PyTorch Dataset 정의
class SimpleDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, idx):
        x = self.data[idx, :-1]  # 입력: 앞 n-1개
        y = self.data[idx, 1:]   # 정답: 다음 토큰
        return x, y

# DataLoader 생성
dataset = SimpleDataset(input_ids)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# 모델 초기화 및 옵티마이저 설정
vocab_size = tokenizer.vocab_size
model = TinyTransformer(vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)

# 간단한 학습 루프 (3 epoch)
model.train()
for epoch in range(3):
    total_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        logits, _ = model(x)  # 예측
        loss = F.cross_entropy(logits.view(-1, vocab_size), y.view(-1))  # next token loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: loss = {total_loss / len(dataloader):.4f}")

# 데모 입력 문장으로 attention 확인
model.eval()
demo_text1 = "Israel England Syria Iran" #이곳에 단어 입력 ---------------------------------------------->
demo_tokens = tokenizer.tokenize(demo_text1)  # 토큰 문자열
# 토크나이징 및 텐서화
demo_inputs = tokenizer(demo_text1, return_tensors="pt").to(device)
with torch.no_grad():
    _, demo_attn = model(demo_inputs["input_ids"])  # 어텐션 가중치 추출

# attention 가중치 표 출력 함수 정의
def print_attention_table(attn_weights, tokens, head=0, batch=0):
    attn = attn_weights[batch, head].detach().cpu().numpy()  # numpy 변환
    df = pd.DataFrame(attn, index=tokens, columns=tokens)  # pandas 표
    print(f"\n🔹 Attention Head {head} Weights:")
    print(df.round(2))  # 소수점 2자리

# 데모 입력에 대한 Head 0의 어텐션 표 출력
print_attention_table(demo_attn, demo_tokens, head=0)

demo_text2 = "Israel England France Iran" #이곳에 단어 입력 --------------------------------------------->
demo_tokens = tokenizer.tokenize(demo_text2)  # 토큰 문자열
# 토크나이징 및 텐서화
demo_inputs = tokenizer(demo_text2, return_tensors="pt").to(device)
with torch.no_grad():
    _, demo_attn = model(demo_inputs["input_ids"])  # 어텐션 가중치 추출

# attention 가중치 표 출력 함수 정의
def print_attention_table(attn_weights, tokens, head=0, batch=0):
    attn = attn_weights[batch, head].detach().cpu().numpy()  # numpy 변환
    df = pd.DataFrame(attn, index=tokens, columns=tokens)  # pandas 표
    print(f"\n🔹 Attention Head {head} Weights:")
    print(df.round(2))  # 소수점 2자리

# 데모 입력에 대한 Head 0의 어텐션 표 출력
print_attention_table(demo_attn, demo_tokens, head=0)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

Epoch 1: loss = 11.0156
Epoch 2: loss = 10.9808
Epoch 3: loss = 10.9460

🔹 Attention Head 0 Weights:
          Israel  ĠEngland  ĠSyria  ĠIran
Israel      0.23      0.20    0.32   0.25
ĠEngland    0.27      0.27    0.15   0.31
ĠSyria      0.27      0.20    0.28   0.25
ĠIran       0.27      0.25    0.25   0.23

🔹 Attention Head 0 Weights:
          Israel  ĠEngland  ĠFrance  ĠIran
Israel      0.24      0.20     0.30   0.26
ĠEngland    0.22      0.22     0.31   0.25
ĠFrance     0.32      0.32     0.20   0.16
ĠIran       0.31      0.28     0.15   0.26
